# Compare human AD Control vs disease

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SeuratWrappers)
    library(patchwork)
    library(ggplot2)
    library(repr)
    library(gridExtra)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(tidyverse)
    library(ggrepel)
    library(scater)
    library(fs)
    library(Signac)
})
options(future.globals.maxSize = 1e9)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
setwd("~/5XFAD_mouse/data/")

In [3]:
seurat_object <- readRDS(
    "humanAD_ATAC_signac_object.rds")

In [4]:
seurat_object <- subset(seurat_object, subset = group %in% c("AD", "Normal"))

In [5]:
subclasses <- path_ext_remove(path_file(dir_ls("humanAD_raw_peaks/")))

In [6]:
table(seurat_object$orig.ident, seurat_object$group)

          
             AD Normal
  ad13     5344      0
  ad1551   3760      0
  ad22     3935      0
  ad24     5018      0
  ad671    6393      0
  hc11        0   1912
  hc1153      0   2550
  hc12        0   5747
  hc1203      0   5430
  hc1216      0   7955
  hc19        0   4825
  hc212191    0   6038
  hc26        0   8917
  hc35        0   7018
  hc40        0   6302
  hc69984     0   4355
  hc73        0   5307
  hc9         0   5972
  hc98        0   9828

In [13]:
for (subclass in subclasses) {
    # if (subclass %in% c("Choroid-plexus", "Endo", "IOL", "PVM", "SMC-Peri", "VLMC")) {
    #     next
    # }
    
    bed_file <- sprintf("humanAD_raw_peaks/%s.bed", subclass)
    bed_table <- read.table(bed_file, sep = "\t", header = FALSE)
    colnames(bed_table) <- c("chrom", "start", "end", "label")
    kept_peaks <- paste(bed_table$chrom, bed_table$start, bed_table$end, sep = "-")
    curr_object <- subset(seurat_object, subset = final_cluster == subclass)
    curr_object <- curr_object[kept_peaks]
    
    curr_sce <- SingleCellExperiment(
        list(counts = LayerData(curr_object, layer = "counts")),
        colData = curr_object@meta.data
    )
    curr_sce_aggr <- aggregateAcrossCells(curr_sce, curr_object$orig.ident)
    
    curr_counts <- counts(curr_sce_aggr)
    Group <- curr_sce_aggr$group
    Group <- factor(Group, levels = c("Normal", "AD"))
    
    curr_dge <- DGEList(counts = curr_counts, group = Group, remove.zeros = TRUE)
    # keep <- filterByExpr(curr_dge, min.count = 2, min.prop = 0.5)
    # curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + Group)
    colnames(curr_design) <- levels(Group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DAR_humanAD//", paste0(subclass, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

Removing 520 rows with all zero counts

Removing 7 rows with all zero counts

Removing 3 rows with all zero counts

Removing 8 rows with all zero counts

Removing 1 rows with all zero counts

Removing 182 rows with all zero counts

Removing 1 rows with all zero counts

Removing 30 rows with all zero counts

Removing 23 rows with all zero counts

